In [30]:
import sys
import math
import os
import re
from collections import Counter
sys.path.insert(0, '..')
from nbmodel import NBClassifier

ImportError: cannot import name 'NBClassifier'

In [31]:
pos_path = os.path.expanduser("~/Dropbox/NLP Readings/hw 2/hotelPosT-rand_training.txt")
neg_path = os.path.expanduser("~/Dropbox/NLP Readings/hw 2/hotelNegT-rand_training.txt")
test_path = os.path.expanduser("~/Dropbox/NLP Readings/hw 2/rand_test_neg_and_pos.txt")
eval_path = os.path.expanduser("~/Dropbox/NLP Readings/hw 2/eval_rand_neg_and_pos.txt")

In [32]:
def file_to_tuples(path, cat):
    """
    Create and normalize document tuples from path.
    
    Returns list of tuples: [(text, cat), ...]
    """
    tuples = []
    with open(path) as f:
        for line in f:
            line = re.split('\t', line)[1:] #ignore first and last element in the list.
            line = line[0].lower()
            line = line.rstrip()
            line = re.sub(r'[^a-z|^0-9|^\s]+', '' , line)
            tuples.append((line, cat))
    return tuples

In [25]:
pos_docs = file_to_tuples(pos_path, 'pos')
neg_docs = file_to_tuples(neg_path, 'neg')

In [19]:
m = NBModel(['pos', 'neg'])
m.train(pos_docs)
m.train(neg_docs)

10749
1727
11743
11502


In [26]:
# make predictions
predictions = {}
with open(test_path) as f:
    for line in f:
        review_id, review = line.split("\t")
        review = review.lower()
        review = review.rstrip()
        review = re.sub(r'[^a-z|^0-9|^\s]+', '' , review)
        cat = m.predict(review)
        predictions[review_id] = cat

In [27]:
# get gold categories
golden = {}
with open(eval_path) as f:
    for line in f:
        review_id, cat = line.split("\t")
        golden[review_id] = cat.lower().rstrip()

In [28]:
def evaluate(gold_dict, pred_dict, beta=1):
    tp, tn, fp, fn = 0, 0, 0, 0
    for review_id, cat in pred_dict.items():
        golden_cat = gold_dict[review_id]
        print(review_id, cat, golden_cat)
        if golden_cat == 'pos':
            if cat == 'pos':
                tp += 1
            else:
                fn += 1
        else:
            if cat == 'pos':
                fp += 1
            else:
                tn += 1
    precision = tp / (tp+fp)
    recall = tp / (tp+fn)
    accuracy = (tp + tn)/(tp+tn+fp+fn)
    f1 = ((beta**2 + 1) * precision * recall) / ((beta**2 * precision) + recall)
    print(tp, tn, fp, fn)
    return (f1, accuracy, precision, recall)

In [29]:
evaluate(golden, predictions)

ID-1088 pos pos
ID-1196 neg pos
ID-0909 neg neg
ID-0921 neg neg
ID-0913 neg neg
ID-1180 pos pos
ID-1124 pos pos
ID-0969 neg neg
ID-0951 neg neg
ID-0967 neg neg
ID-1017 neg neg
ID-1118 pos pos
ID-1262 pos pos
ID-0927 neg neg
ID-1023 neg neg
ID-0833 neg neg
ID-1027 neg neg
ID-0841 neg neg
ID-1266 pos pos
ID-1212 pos pos
ID-1104 pos pos
ID-1114 pos pos
ID-1112 pos pos
ID-0889 neg neg
ID-0871 neg neg
ID-1106 pos pos
ID-1029 neg neg
ID-1136 pos pos
ID-1001 neg neg
ID-0867 neg neg
ID-1150 pos pos
ID-1134 pos pos
ID-0963 pos neg
ID-1230 pos pos
ID-1168 pos pos
ID-0817 neg neg
ID-1007 neg neg
ID-0819 neg neg
16 20 1 1


(0.9411764705882353,
 0.9473684210526315,
 0.9411764705882353,
 0.9411764705882353)